In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = './FMCG/train/images'
csv_data = pd.read_csv('./FMCG/train/_classes.csv')
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [3]:
csv_data.columns

Index(['filename', ' AfterShave', ' FabricBrightener', ' FaceMask',
       ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder',
       ' Soap', ' Sweetner', ' Toothpaste'],
      dtype='object')

In [4]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=csv_data,
    directory=train_dir,
    x_col='filename',
    y_col=[' AfterShave', ' FabricBrightener', ' FaceMask', ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder', ' Soap', ' Sweetner', ' Toothpaste'],  # Keep the spaces
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=True
)

Found 202 validated image filenames.


In [5]:
test_dir = './FMCG/test/images'  # Path to test images
test_csv = './FMCG/test/_classes.csv'  # CSV containing test filenames and labels

test_data = pd.read_csv(test_csv)

In [6]:
test_generator = datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=test_dir,
    y_col=[' AfterShave', ' FabricBrightener', ' FaceMask', ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder', ' Soap', ' Sweetner', ' Toothpaste'],  # Keep the spaces
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',  
    shuffle=False 
)

Found 24 validated image filenames.


In [22]:
import tensorflow as tf
from tensorflow.keras import regularizers

# Define a smaller CNN model with reduced L1 and L2 regularization
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', input_shape=(128, 128, 3)),  # Reduced filters
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),  # Reduced filters
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu', 
                          kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0001)),  # Reduced L1 & L2 regularization
    tf.keras.layers.Dropout(0.3),  # Reduced dropout rate
    tf.keras.layers.Dense(12, activation='sigmoid', 
                          kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0001))  # Reduced L1 & L2 regularization
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with validation data (no early stopping)
history = model.fit(
    train_generator,
    epochs=25
)

C:\Users\MANN\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.0742 - loss: 1.4266
Epoch 2/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1103 - loss: 0.9383
Epoch 3/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1461 - loss: 0.7069
Epoch 4/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.1771 - loss: 0.5943
Epoch 5/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1840 - loss: 0.5278
Epoch 6/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.2241 - loss: 0.4591
Epoch 7/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.2914 - loss: 0.4378
Epoch 8/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3285 - loss: 0.3951
Epoch 9/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3740 - loss: 0.3837
Epoch 10/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3936 - loss: 0.3705
Epoch 11/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4233 - loss: 0.3562
Epoch 12/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5049 - loss: 0.3460
E

In [24]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.4583 - loss: 0.3602
Test Loss: 0.36024370789527893, Test Accuracy: 0.4583333432674408


In [25]:
predictions = model.predict(test_generator)
predicted_classes = predictions.argmax(axis=-1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


In [26]:
class_labels = [' AfterShave', ' FabricBrightener', ' FaceMask', ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder', ' Soap', ' Sweetner', ' Toothpaste']  # Keep the spaces
    

predicted_class_names = [class_labels[predicted_index] for predicted_index in predicted_classes]

for filename, predicted_class in zip(test_data['filename'], predicted_class_names):
    print(f'Filename: {filename}, Predicted Class: {predicted_class}')

Filename: images-1-_jpeg.rf.27fd9e5aa4f310481f20e23895d9d15c.jpg, Predicted Class:  Oil
Filename: download-5-_jpeg.rf.9b69ce4dbbb2058f38d42ec41caeb398.jpg, Predicted Class:  PetroliumJelly
Filename: images_jpeg.rf.6a81f460b00de4513ede32c01948f2fb.jpg, Predicted Class:  Soap
Filename: hq720_jpg.rf.452a8276c4b12dd07d2c472c2b332ddd.jpg, Predicted Class:  Toothpaste
Filename: download_jpeg.rf.b3edaf1cfb9702849cf424e9b5b3ee44.jpg, Predicted Class:  AfterShave
Filename: download-9-_jpeg.rf.9de97aa023aa35128ba21b33160d83ae.jpg, Predicted Class:  Napkin
Filename: 8941100501214_jpg.rf.9e879b8637215a9e7b9a8fcd4ba4a70a.jpg, Predicted Class:  Toothpaste
Filename: chamak-top-img-new-update-2023_png.rf.3519ad81a56c7b9c87895fd3bd19c9f6.jpg, Predicted Class:  Lotion
Filename: OOJ44hHIeX6fowKcTrMoQj56fXi7xkZiRl7FvqkA_jpg.rf.c7ad5a7ce486556eefd9563745559c69.jpg, Predicted Class:  Lotion
Filename: 8def6e4622fca704e114c6785f3013dd_jpg.rf.1de911766e40fe266d60b5df7ae23c13.jpg, Predicted Class:  Lotion
Filen

In [ ]:
model.save('FMCG.h5')